<a href="https://colab.research.google.com/github/dimlish/StarAlgorithm/blob/main/TelemetricStar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
import time

%matplotlib inline

In [2]:
class TelemetricDataset(torch.utils.data.Dataset):
    def __init__(self, data_arr):
        '''
        args:
            data_arr - dataset array
        '''
        self.y = torch.from_numpy(data_arr[:,4:5])
        self.x = torch.from_numpy(np.delete(data_arr, [0, 4, 5], 1))

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [3]:
class BaseRegression(nn.Module):
    '''
    Base regression model
    '''
    def __init__(self):
        super().__init__()
        #self.scale = nn.Parameter(torch.rand(19), dtype=torch.float32)
        self.l1 = nn.Linear(19, 200)
        self.l2 = nn.Linear(200, 500)
        self.l3 = nn.Linear(500, 200)
        self.l4 = nn.Linear(200, 20)
        self.l5 = nn.Linear(20, 1)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        x = self.l5(x)
        return x

In [4]:
class ParallelRegression(nn.Module):
    def __init__(self, BaseClass, *args):
        '''
        args:
            BaseClass - class of models,
                        which will be connected with weights self.l
                        and trained in parallel
            *args - arguments for BaseClass initialization 
        '''
        super().__init__()
        self.m1 = BaseClass(*args)
        self.m2 = BaseClass(*args)
        self.l = nn.Linear(2, 1, bias=False)

    def forward(self, x):
        x = self.l(torch.hstack((self.m1(x), self.m2(x))))
        return x

In [5]:
class AggregationRegression(nn.Module):
    def __init__(self, trained_first, trained_second):
        '''
        args:
            trained_first, trained_second - trained models, which will be aggregated to a bigger model
        '''

        super().__init__()
        self.first = trained_first
        self.second = trained_second
        for p in self.first.parameters():
            p.requires_grad = False
        for p in self.second.parameters():
            p.requires_grad = False
        self.l = nn.Linear(2, 1, bias=False)

    def forward(self, x):
        x = self.l(torch.hstack((self.first(x), self.second(x))))
        return x

In [6]:
class StarRegression(nn.Module):
    def __init__(self, trained_model, BaseClass, *args):
        '''
        args:
            trained_model - trained model (its parameters are freezed)
            BaseClass - class of new model,
                        which will be connected with trained_model
                        and trained alongside weights of models (self.l)
            *args - arguments for BaseClass initialization 
        '''

        super().__init__()
        self.trained = trained_model
        for p in self.trained.parameters():
            p.requires_grad = False
        self.training = BaseClass(*args)
        self.l = nn.Linear(2, 1, bias=False)

    def forward(self, x):
        x = self.l(torch.hstack((self.trained(x), self.training(x))))
        return x

In [7]:
train_ratio = 0.8
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 25
sigma = 1
k = 6
epoches = 150
enable_triple = True # enables triple star model

In [8]:
data = np.genfromtxt("/content/drive/MyDrive/parkinsons_updrs.data",
                     delimiter=',', dtype=np.float32)
train_len = round(train_ratio * len(data))
train_set = TelemetricDataset(data[:train_len,])
test_set = TelemetricDataset(data[train_len:,])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

crit = nn.MSELoss() # criterion

In [48]:
def train(model, trainloader, testloader, epoch_num, lr=0.0001, momentum=0, tol=1e-5, test_freq=2, plot_graph=False, name=None):
    '''
    args:
    model - model to be trained
    trainloader - dataloader of training dataset
    testloader - dataloader of testing dataset
    epoch_num - amount of epoches to train
    lr - learning rate
    momentum - momentum for SGD
    tol - tolerance of loss, if difference is < tol, training is stopped
    test_freq - every test_freq epoches loss on testing dataset is calculated
    plot_graph - if true, then real time graph is shown, else loss is printed
    name - name of model
    
    returns:
      losses - list of avg loss on training dataset per every test_freq epoches
    '''

    start_time = time.time()
    if name is not None:
        print("\n" + '-' * 50)
        print(name + ":")
    losses = []
    optimizer = optim.SGD(model.parameters(), lr, momentum=momentum)

    for epoch in range(epoch_num):
        for x_train, y_train in trainloader:
 #           x_train = x_train.reshape((x_train.shape[0], ))
 #           y_train = y_train.reshape((y_train.shape[0], 1))
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            y_pred = model(x_train)
      
            loss = crit(y_pred, y_train)
            print("x:", x_train)
            print("expected:", y_train)
            print("prediction", y_pred)
            print("loss:", loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        if epoch % test_freq == 0:
            mean_val_loss = []
            val_accuracy = []
            with torch.no_grad():
                for x_test, y_test in testloader:
#                    x_test = x_test.reshape((x_test.shape[0], 1))
#                    y_test = y_test.reshape((y_test.shape[0], 1))
                    x_test = x_test.to(device)
                    y_test = y_test.to(device)
                    y_pred = model(x_test)
                    print("test expected:", y_train)
                    print("test prediction", y_pred)
                    loss = crit(y_pred, y_test)
                    print("test loss:", loss)
                    mean_val_loss.append(loss.numpy())

            losses.append(np.mean(mean_val_loss))


            if plot_graph:
                clear_output(True)
                (x, y) = test_set.getall()
                x = x.reshape((test_size, 1))
                plt.scatter(x, y, c='black', marker='.', label='true')
                y = model(x)
                plt.scatter(x, y.detach().numpy(), marker='.', c='red', label=name)
                plt.legend(loc='upper left')
                plt.title('Epoch: {epoch}, loss: {loss}'.format(
                      epoch=epoch, loss=np.mean(mean_val_loss)))
                plt.show()
            else:
                print('Epoch: {epoch}, loss: {loss}'.format(
                      epoch=epoch, loss=np.mean(mean_val_loss)))

            if losses[-1] != losses[-1]:
                raise ValueError("NaN loss")
            if len(losses) > 2 and abs(losses[-1] - losses[-2]) < tol:
                while len(losses) != epoch_num // test_freq:
                    losses.append(losses[-1]) # fill the list before returning
                break

    print("Training time:", round(time.time() - start_time), "seconds")
    return losses, time.time() - start_time

In [49]:
base = BaseRegression()
base.to(device)

base_losses, base_time = train(base, train_loader, test_loader, epoches, name="base")

Выходные данные были обрезаны до нескольких последних строк (5000).
        [24.6990],
        [22.8560],
        [21.9310],
        [21.0090],
        [20.0910],
        [19.1680],
        [18.2460],
        [27.6820],
        [28.0390]])
prediction tensor([[23.0561],
        [29.6691],
        [29.8601],
        [29.6131],
        [29.7116],
        [29.8279],
        [29.7012],
        [29.4177],
        [28.1823],
        [27.7333],
        [27.2762],
        [26.0728],
        [25.4719],
        [25.1515],
        [24.9692],
        [24.8305],
        [24.3931],
        [23.2738],
        [23.4406],
        [23.2711],
        [23.0765],
        [22.5495],
        [23.0147],
        [29.7233],
        [30.0792]], grad_fn=<AddmmBackward>)
loss: tensor(6.5524, grad_fn=<MseLossBackward>)
x: tensor([[7.5000e+01, 0.0000e+00, 2.7379e+01, 9.1800e-03, 6.9910e-05, 3.4300e-03,
         4.7000e-03, 1.0290e-02, 5.4410e-02, 4.8600e-01, 1.8710e-02, 3.2890e-02,
         7.8610e-02, 5.6140e-02, 2.

ValueError: ignored

In [50]:
for p in base.named_parameters():
  print(p)

('l1.weight', Parameter containing:
tensor([[-0.2305,  0.0561,  0.2116,  ..., -0.0111,  0.1623,  0.1774],
        [-0.2295, -0.2010,  0.2114,  ...,  0.2211, -0.0428, -0.1895],
        [-0.1171, -0.0013, -0.0850,  ...,  0.0404,  0.2272, -0.0272],
        ...,
        [-0.1248,  0.0464, -0.1367,  ..., -0.1224,  0.0336, -0.1898],
        [-0.1577,  0.0690,  0.0345,  ..., -0.1355,  0.0745,  0.0126],
        [ 0.1123,  0.1623, -0.1969,  ..., -0.0824,  0.2142,  0.0762]],
       requires_grad=True))
('l1.bias', Parameter containing:
tensor([ 0.1895, -0.1712,  0.1061, -0.0497, -0.0556, -0.1739,  0.0275, -0.0263,
         0.0454, -0.1601,  0.0046, -0.1335,  0.0515,  0.1383,  0.1537,  0.1521,
        -0.0854, -0.2223,  0.0323,  0.1695,  0.1489,  0.1152,  0.0583, -0.1971,
         0.1765, -0.2289,  0.1102,  0.0730,  0.2028,  0.1968,  0.1341, -0.1068,
        -0.1890,  0.1113,  0.2056,  0.0235, -0.2026, -0.1819, -0.0987,  0.1165,
        -0.0713, -0.1972,  0.2206, -0.0421, -0.2199,  0.1513, -0.225

In [ ]:
parallel = ParallelRegression(BaseRegression)
parallel.to(device)
parallel_losses, parallel_time = train(parallel, train_loader, test_loader, epoches, name="parallel")

In [ ]:
#first = BaseRegression()
#first.to(device)
#agg_time = train(first, train_loader, test_loader, epoches, name="first")[1]
second = BaseRegression()
second.to(device)
agg_time = train(second, train_loader, test_loader, epoches, name="second")[1]
agg = AggregationRegression(base, second)
agg.to(device)
agg_losses, dt = train(agg, train_loader, test_loader, epoches, name="aggregation")
agg_time += dt + base_time
print('Total time:', agg_time)

In [ ]:
star = StarRegression(base, BaseRegression)
star.to(device)
star_losses, star_time = train(star, train_loader, test_loader, epoches, name="star")
print('Total time:', star_time + base_time)

In [ ]:
if enable_triple:
    triple = StarRegression(star, BaseRegression) # star of star and base
    triple_losses = train(triple, train_loader, test_loader, epoches, name="triple")

In [ ]:
for p in star.l.named_parameters():
  print(p) # weights of neural networks in star
if enable_triple:
    for p in triple.l.named_parameters():
      print(p) # weights of neural networks in triple star

In [ ]:
# Let's plot true values and predictions of our models for test dataset

plt.figure(1)

(x, y) = test_set.getall()
x = x.reshape((test_size, 1))
plt.scatter(x, f(x), c='black', label='true')

y = base(x)
plt.scatter(x, y.detach().numpy(), c='r', label='base')

y = parallel(x)
plt.scatter(x, y.detach().numpy(), c='g', label='parallel')

y = agg(x)
plt.scatter(x, y.detach().numpy(), c='violet', label='aggregation')

y = star(x)
plt.scatter(x, y.detach().numpy(), c='b', label='star')

if enable_triple:
    y = triple(x)
    plt.scatter(x, y.detach().numpy(), c='orange', label='triple')

plt.legend(loc='upper left')
plt.show()


# And let's plot losses of corresponding models

indexes = [i for i in range(0, epoches, 2)]

plt.figure(2)
plt.plot(indexes, base_losses, c='r', label='base')
plt.plot(indexes, parallel_losses, c='g', label='parallel')
plt.plot(indexes, agg_losses, c='violet', label='aggregation')
plt.plot(indexes, star_losses, c='b', label='star')
if enable_triple:
    plt.plot(indexes, triple_losses, c='orange', label='triple')
plt.legend(loc='upper left')
plt.show()
